# 데이터 분석 주제 선정
- 데이터를 분석하려는 목적 설정
- 목적에 부합하는 데이터를 제공하는 사이트 조사

# 데이터 수집을 위한 설계
- 사이트로부터 어떤 데이터를 수집할 것인지 파악하고 수집 후 CSV파일로 저장하기 위한 설계
    ㄴ 단순 파일에 저장
    ㄴ 데이터 베이스를 활용하여 저장
    
# 분석을 위한 데이터 수집
- 조사한 사이트들을 분담하여 Crawling 또는 Scraping을 활용하여 데이터 수집
- 수집한 데이터를 앞서 설계한 구조로 저장
- 저장된 데이터 확인

# (2023.6.26)
- for 반복문을 통해 제목으로 들어간다. 
- 책의 제목, 저자, 장르, 평점, 가격, 발행일자를 가져와 리스트에 담는다.
- 영풍문고의 경우, 발행번호가 뜨지 않아 ( / ) 로 반복.
- 담는 것이 완료되면 .back() 을 통해 베스트셀러의 메인으로 나온다.
- ( 영풍 ) 베스트셀러가 1위와 2-4위, 5위~ 의 스크래핑 코드가 서로 다르므로,
- if 제어문으로 이미 사용한 코드는 패스한 후, 새로 받아오는 코드로 이동, 위의 순서를 반복시켜준다.

# (2023.6.27 수정)
- 이전의 정보는 selector, xpath 의 코드가 전부 다르므로 철회.
- 사이트가 주간베스트셀러 사이트 밖에 없으므로, 월간베스트셀러 사이트로 이동하는 코드 작성.
- for 반복문을 활용하여 1 ~ 13을 반복.
- 각 함수들이 False 인 경우, 맨 밑 else 에서 순서대로 년, 월, 더보기 클릭하는 코드 작성.
- 2중 for 반복문으로 나오는 책 정보의 개발자 툴을 활용하여 스크래핑, 크롤링해온다.
- 받아온 정보를 리스트에 담고, 판다스를 활용하여 정리, csv 파일에 담는 코드 작성.

Colume : 번호(ISBN) | 제목 | 장르 | 저자(지은이) |  평점(0.1~10.0) | 가격(판매가) | ( 발행일자 )

In [136]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import requests, time, os
import numpy as np
import pandas as pd

chrome = Chrome()
url = 'https://www.ypbooks.co.kr/book_arrange.yp?targetpage=book_week_best&pagetype=5&depth=1'
chrome.get(url)

time.sleep(1)
month_btn = chrome.find_element(By.XPATH, value='//*[@id="DateSearch"]/div/ul/li[2]/a')
month_btn.click()

def year_month():
        
    cnt = 6
    num = 2022
    k = 1
    
    for i in range(1, 13):
        
        book_li = []
        
        if i == 7:
            cnt = 0
            num += 1
            
        else:
            pass
        
        if i == 13:
            num += 1
            
        else:
            pass
        
        if num == 2024:
            break
        
        else:
            cnt += 1
            time.sleep(1)
            select_year = Select(chrome.find_element(By.ID, 'yearSelect'))
            select_year.select_by_value(f'{num}')

            time.sleep(1)
            select_month = Select(chrome.find_element(By.ID, 'monthSelect'))
            select_month.select_by_value(f'{cnt}')
            
            time.sleep(1)
            search_btn = chrome.find_element(By.XPATH, value='//*[@id="reSearchSubmit"]')
            search_btn.click()
            
            data = chrome.page_source
            page = BeautifulSoup(data)
            new_page = page.find_all('a', {'class' : 'btn type01'})
            
            for list in new_page :

                code = list.attrs['href'][-11:-2]

                base_url = 'https://www.ypbooks.co.kr/ypbooks/book/popup/pop_book_info.jsp?pBookCode='
                new_url = base_url + code
                
                book = requests.get(new_url)
                bs = BeautifulSoup(book.text, 'html.parser')

                sel_isbn = '#bookInfo > li:nth-child(3)'
                isbn = bs.select(sel_isbn)[0].text
                
                sel_title = '#bookTitle > h2'
                title = bs.select(sel_title)[0].text
                
                sel_genre = '#detailInfoContents > p'
                genre = bs.select(sel_genre)[0].text.strip()
                
                sel_writer = '#bookTitle > p > span:nth-child(1)'
                writer = bs.select(sel_writer)[0].text
                
                point = '/'
                
                sel_price = '#priceInfo > li:nth-child(2) > span.afterDC'
                price = bs.select(sel_price)[0].text + '원'
                
                sel_date = '#bookInfo > li:nth-child(1)'
                date = bs.select(sel_date)[0].text
                
                book_li.append([isbn[11:24], title, genre, writer, point, price, date[6:19]])
                            
            li_index = ['ISBN', '책 제목', '도서 장르', '저자', '평점', '판매가격', '발행일']

            index_df = pd.DataFrame(columns=li_index)
            index_df.loc[0] = ['', '', '', '', '', '', '']
            index_df.index = [str(num) + '년' + ' ' + str(cnt) + '월']

            book_df = pd.DataFrame(np.array(book_li), columns=li_index)
            book_df.index = range(1, len(book_df) + 1)
            book_df = index_df.append(book_df)
            
            print(book_df)
        
            if not os.path.exists('best_seller2.csv') :
                book_df.to_csv('best_seller2.csv', encoding='CP949', mode = 'w')
            
            else :
                book_df.to_csv('best_seller2.csv', encoding='CP949', mode = 'a', header=False)
            
            print('저장 완료')
        
year_month()

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)


                  ISBN                            책 제목 도서 장르             저자  \
2022년 7월                                                                      
1         791161571188                 불편한 편의점(벚꽃 에디션)  국내도서            김호연   
2         791191043754             세상의 마지막 기차역-리커버 에디션  국내도서         무라세다케시   
3         788901260716                             역행자  국내도서             자청   
4         791168412279                          에그박사 7  국내도서       에그박사,박송이   
5         791191114225                            작별인사  국내도서            김영하   
6         791168411685                         흔한남매 10  국내도서       흔한남매,백난도   
7         791191043723              어느 날 내 죽음에 네가 들어왔다  국내도서          세이카료겐   
8         791133463817             포켓몬스터 썬&문 포켓몬 전국대도감  국내도서            편집부   
9         791197377143       어서 오세요 휴남동 서점입니다 (봄꽃 에디션)  국내도서            황보름   
10        791164064410                이상한 과자 가게 전천당 15  국내도서        히로시마레이코   
11        788950903138          마법천자문 53-결정을 내려라 결단할

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)


                  ISBN                                 책 제목 도서 장르  \
2022년 8월                                                            
1         791168412477                              흔한남매 11  국내도서   
2         788954699914                                  하얼빈  국내도서   
3         791161571188                      불편한 편의점(벚꽃 에디션)  국내도서   
4         791161571379                     불편한 편의점2(단풍 에디션)  국내도서   
5         788901260716                                  역행자  국내도서   
6         791191043754                  세상의 마지막 기차역-리커버 에디션  국내도서   
7         791168340510                            파친코 1-개정판  국내도서   
8         791133463817                  포켓몬스터 썬&문 포켓몬 전국대도감  국내도서   
9         791191043761            멘탈을 바꿔야 인생이 바뀐다(10만부 리커버)  국내도서   
10        791197377143            어서 오세요 휴남동 서점입니다 (봄꽃 에디션)  국내도서   
11        791191043297              오늘 밤 세계에서 이 사랑이 사라진다 해도  국내도서   
12        791158883591           부자아빠 가난한 아빠 1(20주년 특별 기념판)  국내도서   
13        791168412248  흔한남매 이상한 나

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)


...

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)


                  ISBN                                           책 제목 도서 장르  \
2023년 4월                                                                      
1         791168473690                                       세이노의 가르침  국내도서   
2         791169791977                                       스즈메의 문단속  국내도서   
3         791197647666                                  기분이 태도가 되지 말자  국내도서   
4         788997575169                     원씽(THE ONE THING)(리커버 특별판)  국내도서   
5         791168415157                             에그박사 9-자연 생물 관찰 만화  국내도서   
6         791161571188                                불편한 편의점(벚꽃 에디션)  국내도서   
7         791188469994        당신은 결국 무엇이든 해내는 사람(10만 부 기념 특별 리커버 에디션)  국내도서   
8         791168415232                                        흔한남매 13  국내도서   
9         791169792035                스즈메의 문단속 (양장)(신카이 마코토 하드커버 시리즈)  국내도서   
10        791169794879               타키 포오 코믹 어드벤처 1 - 캐릭온TV 오리지널 코믹스  국내도서   
11        791198013026                김미경의 마흔 수업(이미 

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)


                  ISBN                                     책 제목 도서 장르  \
2023년 5월                                                                
1         791168473690                                 세이노의 가르침  국내도서   
2         791168415232                                  흔한남매 13  국내도서   
3         791169791977                                 스즈메의 문단속  국내도서   
4         791188331888                                    사장학개론  국내도서   
5         791192504216                     백앤아 1 - 미스터리 100층 감옥  국내도서   
6         791169794879         타키 포오 코믹 어드벤처 1 - 캐릭온TV 오리지널 코믹스  국내도서   
7         791197647666                            기분이 태도가 되지 말자  국내도서   
8         791192579504                              바다가 들리는 편의점  국내도서   
9         791168415171      슈뻘맨의 숨은 과학 찾기 1 - 엉뚱 도전 속에 숨은 과학 상식  국내도서   
10        788997575169               원씽(THE ONE THING)(리커버 특별판)  국내도서   
11        791161571188                          불편한 편의점(벚꽃 에디션)  국내도서   
12        791188469994  당신은 결국 무엇이든 해내는 사람(10만 부 기념

C:\Users\jongyeon.woo\AppData\Local\Temp\ipykernel_14972\993516176.py:114: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  book_df = index_df.append(book_df)
